In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install openai==0.28

In [3]:
import openai
print(openai.__version__)


0.28.0


In [4]:
import os
openai.api_key = os.getenv("OPENAI_API_KEY")

### Write prompts using OPENAI API keys that perform the Math task, PDF -website Reading and display the output ###

In [5]:
!pip install PyPDF2


In [15]:
import requests
from bs4 import BeautifulSoup
import PyPDF2
import openai

In [6]:
# Extract website content
def extract_website_content(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, "html.parser")
            paragraphs = soup.find_all("p")
            content = " ".join([para.get_text() for para in paragraphs])
            return content.strip()
        else:
            return f"Failed to fetch website content. Status code: {response.status_code}"
    except Exception as e:
        return f"Error fetching website: {e}"

# Extract PDF content
def extract_pdf_content(pdf_path):
    try:
        with open(pdf_path, "rb") as pdf_file:
            reader = PyPDF2.PdfReader(pdf_file)
            text = ""
            for page in reader.pages:
                text += page.extract_text()
        return text.strip()
    except Exception as e:
        return f"Error reading PDF: {e}"

# Rewrite content using OpenAI
def rewrite_content(content, model="gpt-3.5-turbo"):
    try:
        response = openai.ChatCompletion.create(
            model=model,
            messages=[
                {"role": "system", "content": "You are a content writer specializing in simplifying text."},
                {"role": "user", "content": f"Rewrite this content for better readability:\n\n{content}"}
            ],
            max_tokens=1500,
            temperature=0.7
        )
        return response['choices'][0]['message']['content'].strip()
    except Exception as e:
        return f"Error rewriting content: {e}"

# Solve math problems
def solve_math_problem(problem, model="gpt-3.5-turbo"):
    try:
        response = openai.ChatCompletion.create(
            model=model,
            messages=[
                {"role": "system", "content": "You are a math tutor who explains problems step-by-step."},
                {"role": "user", "content": f"Solve this math problem step-by-step: {problem}"}
            ],
            max_tokens=500,
            temperature=0.5
        )
        return response['choices'][0]['message']['content'].strip()
    except Exception as e:
        return f"Error solving math problem: {e}"

# Summarize the conversation
def summarize_chat(messages, model="gpt-3.5-turbo"):
    try:
        chat_content = "\n".join(messages)
        response = openai.ChatCompletion.create(
            model=model,
            messages=[
                {"role": "system", "content": "You are a summarization assistant."},
                {"role": "user", "content": f"Summarize this chat session:\n\n{chat_content}"}
            ],
            max_tokens=300,
            temperature=0.5
        )
        return response['choices'][0]['message']['content'].strip()
    except Exception as e:
        return f"Error summarizing chat: {e}"

# Workflow


# Que 1: Solve a math problem using OpenAi

In [9]:

math_problem = "Find the value of x if 3x + 5 = 20."
print("Solving math problem...")
math_solution = solve_math_problem(math_problem)
print("\nMath Problem Solution:\n", math_solution)

Solving math problem...

Math Problem Solution:
 Step 1: Subtract 5 from both sides of the equation to isolate the term with x on one side.
3x + 5 - 5 = 20 - 5
3x = 15

Step 2: Divide both sides by 3 to solve for x.
3x/3 = 15/3
x = 5

Therefore, the value of x is 5.


# Que 2: Process PDF content
- Give a PDF and ask the model to rewrite and answer questions on the given PDF.

In [10]:

pdf_path = "/content/DeepLearning.pdf"
print("\nExtracting DeepLearning PDF content...")
pdf_content = extract_pdf_content(pdf_path)


Extracting DeepLearning PDF content...


# Rewritten pdf content #

In [11]:
if pdf_content:
    print("\nExtracted DeepLearning PDF Content (Preview):\n", pdf_content[:500], "...\n")
    rewritten_pdf_content = rewrite_content(pdf_content)
    print("\nRewritten PDF Content:\n", rewritten_pdf_content)
else:
    print("No content extracted from the PDF.")


Extracted DeepLearning PDF Content (Preview):
 DDoSNet: A Deep-Learning Model for
Detecting Network Attacks
Mahmoud Said Elsayed1, Nhien-An Le-Khac1, Soumyabrata Dev1,2, and Anca Delia Jurcut1
1School of Computer Science, University College Dublin, Dublin, Ireland
2ADAPT SFI Research Centre, Dublin, Ireland
Abstract —Software-Deﬁned Networking (SDN) is an emerging
paradigm, which evolved in recent years to address the weaknesses in
traditional networks. The signiﬁcant feature of the SDN, which is achievedby disassociating the control plane f ...


Rewritten PDF Content:
 Title: DDoSNet: A Deep-Learning Model for Detecting Network Attacks

Authors: Mahmoud Said Elsayed, Nhien-An Le-Khac, Soumyabrata Dev, Anca Delia Jurcut
Affiliations: School of Computer Science, University College Dublin, Dublin, Ireland; ADAPT SFI Research Centre, Dublin, Ireland

Abstract:
Software-Defined Networking (SDN) has emerged as a new paradigm to address weaknesses in traditional networks by separating the 

# Que 3 : Rewrite the content of the website
- Give a PDF and ask the model to rewrite and answer questions on the given PDF.

In [12]:
url = "https://www.geeksforgeeks.org/introduction-deep-learning/"
print("\nExtracting website content...")
web_content = extract_website_content(url)

if web_content:
    print("\nExtracted Website Content (Preview):\n", web_content[:500], "...\n")
    rewritten_web_content = rewrite_content(web_content)
    print("\nRewritten Website Content:\n", rewritten_web_content)
else:
    print("No content extracted from the website.")


Extracting website content...

Extracted Website Content (Preview):
 In the fast-evolving era of artificial intelligence, Deep Learning stands as a cornerstone technology, revolutionizing how machines understand, learn, and interact with complex data. At its essence, Deep Learning AI mimics the intricate neural networks of the human brain, enabling computers to autonomously discover patterns and make decisions from vast amounts of unstructured data. This transformative field has propelled breakthroughs across various domains, from computer vision and natural lang ...


Rewritten Website Content:
 In today's rapidly evolving world of artificial intelligence, Deep Learning is a key technology that is transforming how machines process complex data. Essentially, Deep Learning AI imitates the intricate neural networks of the human brain, enabling computers to autonomously identify patterns and make decisions from large amounts of unstructured data. This innovative field has led to breakthr

# Summarize the chat

In [14]:
# Step 4: Summarize the conversation
messages = [
    f"Math Problem Solution: {math_solution}",
    f"Rewritten Deep Learning PDF Content: {rewritten_pdf_content}",
    f"Rewritten Deep Learning Website Content: {rewritten_web_content}"
]
print("\nSummarizing the chats...\n")
summary = summarize_chat(messages)
print("\nChat Summary is as below..:\n", summary)



Summarizing the chats...


Chat Summary is as below..:
 The chat session discussed a math problem solution where x was found to be 5. The remaining content focused on a research paper titled "DDoSNet: A Deep-Learning Model for Detecting Network Attacks" which introduces an intrusion detection system based on Deep Learning for improved DDoS attack detection in Software-Defined Networking (SDN) environments. The paper highlights the methodology, results, and implications of using Deep Learning techniques in enhancing network security. Additionally, the rewritten content provides an overview of Deep Learning, its applications, scope, challenges, and the transformative impact it has on various industries.
